In [5]:
import pandas as pd
import numpy as np


In [7]:
#Load translated topics and content datasets
content = pd.read_csv("content_translated_final.csv")
topics = pd.read_csv("topics_final.csv")

In [11]:
# replace NaN values with an empty string
content['title'].fillna('', inplace=True)
content['description'].fillna('', inplace=True)
content['text'].fillna('', inplace=True)
content['kind'].fillna('', inplace=True)

topics['title'].fillna('', inplace=True)
topics['description'].fillna('', inplace=True)

topics representation (title, description, and title of it's parent, and grandparent)

In [12]:
def generate_combined_text(topics):
    combined_texts = []

    for index, row in topics.iterrows():
        child = row
        parent_id = child['parent']

        child_text = f"{child['title']} {child['description']}"

        if parent_id != 'NaN':
            parent = topics[topics['id'] == parent_id]

            if not parent.empty:
                parent_title = parent['title'].values[0]
                child_text += f" {parent_title}"

                grandparent_id = parent['parent'].values[0]
                if grandparent_id != 'NaN':
                    grandparent = topics[topics['id'] == grandparent_id]

                    if not grandparent.empty:
                        grandparent_title = grandparent['title'].values[0]
                        child_text += f" {grandparent_title}"
                        
                        greatgrandparent_id = grandparent['parent'].values[0]
                        if greatgrandparent_id != 'NaN':
                            greatgrandparent = topics[topics['id'] == greatgrandparent_id]

                            if not greatgrandparent.empty:
                                greatgrandparent_title = greatgrandparent['title'].values[0]
                                child_text += f" {greatgrandparent_title}"

        combined_texts.append(child_text)

    return combined_texts

In [13]:
topics['combined_text'] = generate_combined_text(topics)

content representation

In [17]:
def preprocess_content_text(content):
    content_text = content['title'] + ' ' + np.where(
        (content['description'] == ''),
        content['text'] + ' ' + content['kind'],
        content['description'] + ' ' + content['kind']
    )
    return content_text

In [18]:
content['combined_text'] = preprocess_content_text(content)

In [20]:
# keep topics where has_content is True
topics = topics[topics['has_content'] == True].reset_index(drop=True)

In [22]:
# save topics and content
topics.to_csv('topics_doc2vec.csv', index=False)
content.to_csv('content_doc2vec.csv', index=False)